keywords: NLP (TALN), classification.

# The Spam Detector

# Description

Madame Esposito développe pour son entreprise un chatbot dans le but de répondre automatiquement à ses nombreux clients. Cependant son programme reçoit un grand nombre de message malveillant ou à caractère publicitaire ce qui dégrade les performance de son bot en plus d'occasionner des traitements informatique se répercutant sur sa facture d'électricité.

Madame Esposito vous a contacté afin de créer un programme capable de détecter automatiquement les SPAM. 
Pour cela, elle a construit un jeux de donnée comportant un ensemble de SMS de type SPAM et NON SPAM (HAM), disponible à l'adresse suivante : 
http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

Par ailleurs, afin d'intégrer les résultats dans son équipe, elle nous demande les choses suivantes:
* Afin d'estimer le cout du développement et suivre le projet, une checklist des taches à réaliser doit être rédigé.
* Vous devez créer des fonctions pour les différentes partie de votre code afin de pouvoir les réutiliser facilement
* Vous devez effectuer une validation croisé (cross-validation) sur 10 jeux d'apprentissage et de test différent. Le seed doit être fixé à 42 et le jeux de test doit représenter 20% des données.
* Comparer au moins trois algorithme de classification en terme de **f1 score**. Lequel est le plus puissant ?

## Bonus


Pouvez améliorer les résultats ?
* est-ce que la lemmatisation améliore les résultats ?
* est-ce que la racinisation (stemming) améliore les résultats ?

* est-ce que la lemmatisation améliore les résultats ?
* est-ce que la racinisation (stemming)  améliore les résultats ?

Une fois ces étapesréalisées, reproduire la même expérience avec le jeux suivant, représentant cette fois des commentaires Youtube :
https://archive.ics.uci.edu/ml/datasets/YouTube+Spam+Collection

Les performances sont-elles similaire à ceux obtenus avec le jeux de données précèdent ?

Madame esposito souhaite contrôler si les modèles appris avec le premier jeux de données sont capable de prédire les données de test du deuxième jeu et vice-versa.
Réaliser un tableau comparant
* les résultats de prédiction du modelés appris sur les SPAN SMS pour prédire les SPAM commentaire Youtube.
* et les résultats de prédiction du modèles appris sur les SPAM commentaire Youtube pour prédire les SPAM SMS.



# Proposed Plan

**1) Veille en Traitement du langage + checklist+45min**

https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32

https://towardsdatascience.com/your-guide-to-natural-language-processing-nlp-48ea2511f6e1

https://code.tutsplus.com/fr/tutorials/introducing-the-natural-language-toolkit-nltk--cms-28620

https://towardsdatascience.com/introduction-to-natural-language-processing-for-text-df845750fb63

* parsing and tokenization ?
* vectorization ? Bag of words ?
* advantage of TFIDF ?
* stops words ?

**2) Load, clean and prepare the data**

Vectoriser et netoyer vos données.
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

Quelle est le type Python de vos données après vectorisation.

* Quelle sont les 10 mots les plus fréquent dans le jeux de données ?
* Les moins fréquents ?
* Tracer la distribution de la fréquence des mots présents dans le jeux de données. Qu'observez vous ?




**Aides/propositions**
* Stop words avec NLTK
* scikit-learn pour la vectorization

**3) Apprenstissage**

Que représente la mesure f1 ?
Quelles sont ces avantages sur d'autre mesure tel que la précision le rappel ou l'accuracy ?

**3) Veille cross validation**
https://scikit-learn.org/stable/modules/cross_validation.html

Utilisez la méthode de [ShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit) pour construire vos jeux de données permettant la validation croisée.

**4) Train**

Fit the models and compare the performance in a table that show
* the mean of the f1 score
* the standard deviation of the f1 score

Qu'observez vous ?




In [234]:
import nltk
nltk.download()
# L'intégralité des bibliothèques installées sauf pe08

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## **1) Veille en Traitement du langage + checklist+45min**

VOIR "COURS.ipyjp"

## **2) Load, clean and prepare the data**

In [230]:
import pandas as pds

# ouverture du fichier
sms=pds.read_csv("./data/SMSSpamCollection.txt",sep='\t',names=["ham_or_spam","texte"])
# on remplace ham et spam par des valeurs booléennes
sms['ham_or_spam'] = sms['ham_or_spam'].replace({'ham':1, 'spam':0})
print(sms)
ham=sms[sms['ham_or_spam']== True]
spam=sms[sms['ham_or_spam']== False]
print("sur {} sms, il y a {} ham et {} spam.".format(len(sms),len(ham), len(spam)))

      ham_or_spam                                              texte
0               1  Go until jurong point, crazy.. Available only ...
1               1                      Ok lar... Joking wif u oni...
2               0  Free entry in 2 a wkly comp to win FA Cup fina...
3               1  U dun say so early hor... U c already then say...
4               1  Nah I don't think he goes to usf, he lives aro...
...           ...                                                ...
5567            0  This is the 2nd time we have tried 2 contact u...
5568            1               Will ü b going to esplanade fr home?
5569            1  Pity, * was in mood for that. So...any other s...
5570            1  The guy did some bitching but I acted like i'd...
5571            1                         Rofl. Its true to its name

[5572 rows x 2 columns]
sur 5572 sms, il y a 4825 ham et 747 spam.


### **Vectorisation des données**

#### **Vectorisation simple sans filtration**

In [231]:
import pandas as pds
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#-------------------------------- Vectorisation ---------------------------------------------------
# X est une matrice "compressée". En indice un tuple indiquant (ligne, colonne). La ligne est le 
#  numéro du sms, les colonnes sont les mots utilisés. Le nombre qui aparaît à la ligne x, colonne
#  y est le nombre de fois ou le mots aparaît dans le sms.
#  exemple: (5571, 4253) 2 signifie que dans le sms 5571, le mots numéro 4253 (its) apparaît 2 fois.
# Lorsqu'on utilise la méthode TFID, un flottant inférieur à 1 remplace le nombre d'occurence.
# Le nombrend'occurence est pondérer par le nombre de fois ou le mots apparait dans les sms.
#--------------------------------------------------------------------------------------------------

vectorisation = CountVectorizer()
Xbrut=vectorisation.fit_transform(sms["texte"])
print("Type de donnée retourner par la vectorisation:",type(Xbrut))
print("Taille de la matrice si elle n'était pas condensé:",Xbrut.shape)
print("Le sms 5571 est :",sms.texte[5571])
print("Le mot apparaissant 2 fois dans le sms 5571 est:", vectorisation.get_feature_names()[4253])

print("Résultat de la vectorisation:\n",Xbrut)

print("La liste des mots est donnée par la fonction get_feature_names():\n",vectorisation.get_feature_names())

# On observe que dans cette vectorisation il y a un des numéros (dont beaucoup de numéro de téléphone).
#  Il faudra vérifier à conserver ces "mots numéros" car ils ont une signification.
#print(vectorisation )


Type de donnée retourner par la vectorisation: <class 'scipy.sparse.csr.csr_matrix'>
Taille de la matrice si elle n'était pas condensé: (5572, 8713)
Le sms 5571 est : Rofl. Its true to its name
Le mot apparaissant 2 fois dans le sms 5571 est: its
Résultat de la vectorisation:
   (0, 3571)	1
  (0, 8084)	1
  (0, 4374)	1
  (0, 5958)	1
  (0, 2338)	1
  (0, 1316)	1
  (0, 5571)	1
  (0, 4114)	1
  (0, 1767)	1
  (0, 3655)	1
  (0, 8548)	1
  (0, 4501)	1
  (0, 1765)	1
  (0, 2061)	1
  (0, 7694)	1
  (0, 3615)	1
  (0, 1082)	1
  (0, 8324)	1
  (1, 5538)	1
  (1, 4537)	1
  (1, 4342)	1
  (1, 8450)	1
  (1, 5567)	1
  (2, 4114)	1
  (2, 3373)	1
  :	:
  (5570, 4245)	1
  (5570, 8371)	1
  (5570, 1097)	1
  (5570, 4642)	1
  (5570, 7089)	1
  (5570, 3323)	1
  (5570, 7674)	1
  (5570, 1451)	1
  (5570, 5367)	1
  (5570, 2606)	1
  (5570, 8120)	1
  (5570, 1794)	1
  (5570, 7099)	1
  (5570, 2905)	1
  (5570, 3489)	1
  (5570, 1802)	1
  (5570, 3709)	1
  (5570, 4188)	1
  (5570, 914)	1
  (5570, 1561)	1
  (5571, 7806)	1
  (5571, 5

#### **Fitration par utilisation des "stops words"**  
Il serait possible **d'ajouter** à la liste **d'autres "stops words"** en étudiant les mots contenus dans X.
Pour plus d'efficacité il faudrait aussi faire une recherche des **numéros de téléphone** (qui sont en nombre important). Certain sont écrit sous la forme xxxxxxxxxx, d'autres sous la forme xx xx xx xx xx, d'autre encore sont sous d'autre format. L'utilisation des regex pourrait aider à cette tâche.  

On pourrait également **s'interessé au synonyme** (par exemple ok, OK, oki, okey,...) **et les regrouper** pour diminuer la liste de mots et augmenter la pertinance de l'étude.


In [238]:
#-------------------------------------------------------------------------------------------------
#
# UTILISATION DE STOPWORDS
#
#-------------------------------------------------------------------------------------------------

import pandas as pds
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# stopwords classique
#print(nltk.corpus.stopwords.words('english'))

# Chargement des données 
textesms=list(sms.texte)
validsms=sms['ham_or_spam']
#print(validsms)
#print(sms.ham_or_spam)
stopwords=nltk.corpus.stopwords.words('english')
#print(stopwords)
print("Nombre de mots contenus dans le stopwords",len(stopwords))

vectorisation = CountVectorizer(stop_words=stopwords)
Xstpwrd=vectorisation.fit_transform(textesms)
#print(vectorisation.get_params()) sans grand intérêt
print("Taille de la matrice (en ayant retiré les stopwords) si elle n'était pas condensé:",Xstpwrd.shape)
print("136 mots ont été retiré.")

print("La liste des mots est donnée par la fonction get_feature_names():\n",vectorisation.get_feature_names())

# AJOUTER ICI DECOMPTE DES MOTS

Nombre de mots contenus dans le stopwords 179
Taille de la matrice (en ayant retiré les stopwords) si elle n'était pas condensé: (5572, 8577)
136 mots ont été retiré.
La liste des mots est donnée par la fonction get_feature_names():
 ['00', '000', '000pes', '008704050406', '0089', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '078498', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986030', '080

In [240]:
#-------------------------------------------------------------------------------------------------
#
# UTILISATION DE STOPWORDS ET DE LA PONDÉRATION DES MOTS EN FONCTION DE LEURS FRÉQUENCES DANS LES DOCS
#
#-------------------------------------------------------------------------------------------------

import pandas as pds
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Utilisation des stopwords.
stopwords=nltk.corpus.stopwords.words('english')
#print(stopwords)
print("Nombre de mots contenus dans le stopwords",len(stopwords))

# Pondération des mots avec leurs fréquences d'apparition dans les documents ==> utilisation de
#  TfidfVectorizer
vectorisation = TfidfVectorizer(stop_words=stopwords)
Xstpwrd= vectorisation.fit_transform(textesms)
print(Xstpwrd)

#print(vectorisation.get_params()) sans grand intérêt
print("Taille de la matrice (en ayant retiré les stopwords) si elle n'était pas condensé:",Xstpwrd.shape)
print("136 mots ont été retiré.")

print("La liste des mots est donnée par la fonction get_feature_names():\n",vectorisation.get_feature_names())


Nombre de mots contenus dans le stopwords 179
  (0, 8207)	0.19367133232870914
  (0, 1074)	0.34662215358656684
  (0, 3575)	0.16249778391503625
  (0, 2036)	0.2928277960456836
  (0, 1743)	0.3308883126306653
  (0, 4437)	0.2928277960456836
  (0, 8417)	0.23446293536343882
  (0, 3615)	0.1914742686221826
  (0, 1745)	0.2928277960456836
  (0, 1301)	0.2592620666240387
  (0, 2312)	0.2684349975703689
  (0, 5871)	0.2711122263759042
  (0, 4311)	0.34662215358656684
  (0, 3531)	0.15700111684745632
  (1, 5488)	0.5466243141314314
  (1, 8322)	0.43162957585464123
  (1, 4279)	0.5236804332035243
  (1, 4473)	0.4083258549263009
  (1, 5461)	0.2718944069420321
  (2, 77)	0.23481398547774926
  (2, 1158)	0.16878205227806695
  (2, 6197)	0.16878205227806695
  (2, 7876)	0.12429583820628914
  (2, 7185)	0.1966389668741482
  (2, 6145)	0.1787234410397601
  :	:
  (5567, 5714)	0.20749813661767133
  (5567, 6197)	0.22852426188600214
  (5568, 2959)	0.6485991737677518
  (5568, 3322)	0.5597098620657655
  (5568, 3544)	0.364236977

## **3) Apprenstissage**  

Que représente la mesure f1 ? Quelles sont ces avantages sur d'autre mesure tel que la précision le rappel ou l'accuracy ?

In [241]:
#-------------------------------------------------------------------------------------------------
#
# CLASSIFICATION AVEC UTILISATION DE STOPWORDS ET DE LA PONDÉRATION DES MOTS EN FONCTION DE LEURS
#  FRÉQUENCES DANS LES DOCS
#
#-------------------------------------------------------------------------------------------------
import numpy as np
import pandas as pds
import nltk
#import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn import svm
from sklearn.linear_model import LogisticRegression

# Chargement des données 
textesms= sms.texte
validsms=[1 if x== False else 0 for x in list(sms["ham_or_spam"])]

# Utilisation des stopwords.
stopwords=nltk.corpus.stopwords.words('english')

# Pondération des mots avec leurs fréquences d'apparition dans les documents ==> utilisation de
#  TfidfVectorizer
vectorisation = TfidfVectorizer(stop_words=stopwords)
Xstpwrd= vectorisation.fit_transform(textesms)
#print(Xstpwrd)
#print("La liste des mots est donnée par la fonction get_feature_names():\n",vectorisation.get_feature_names())


# On split le jeux de données
X_train, X_test, Y_train, Y_test = train_test_split(Xstpwrd, validsms, test_size=0.2, random_state=42)
#print(X_train)
#print(Y_train)

In [242]:
#----------------------------------------------------------------------------------------------------------
#
# Modèle de classification SVC
#
#----------------------------------------------------------------------------------------------------------

classification= svm.LinearSVC()
mod_svc= classification.fit(X_train,Y_train)

predict_svc = mod_svc.predict(X_test)
print("score:",mod_svc.score(X_test, Y_test)*100)
print("Coef:",mod_svc.coef_)
print("Matrice:\n", metrics.confusion_matrix(Y_test,predict_svc))
print(classification_report(Y_test,predict_svc))


score: 98.11659192825111
Coef: [[ 0.47655386  0.78928548 -0.020744   ... -0.00828705  0.16700289
   0.        ]]
Matrice:
 [[965   1]
 [ 20 129]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.87      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [243]:
#----------------------------------------------------------------------------------------------------------
#
# Modèle de classification LogisticRegression
#
#----------------------------------------------------------------------------------------------------------

classification= LogisticRegression()
mod_lr= classification.fit(X_train,Y_train)

predict_lr = mod_lr.predict(X_test)

print("score:",mod_lr.score(X_test, Y_test)*100)
print("Coef:",mod_lr.coef_)
print("Matrice:\n", metrics.confusion_matrix(Y_test,predict_lr))

print(classification_report(Y_test,predict_lr))


score: 95.87443946188341
Coef: [[ 0.65420025  1.30795534 -0.01767486 ... -0.01468873  0.13930259
   0.        ]]
Matrice:
 [[965   1]
 [ 45 104]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       0.99      0.70      0.82       149

    accuracy                           0.96      1115
   macro avg       0.97      0.85      0.90      1115
weighted avg       0.96      0.96      0.96      1115



In [244]:
#----------------------------------------------------------------------------------------------------------
#
# Modèle de classification MLPClassifier
#
#----------------------------------------------------------------------------------------------------------
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

#X, y = make_classification(n_samples=100, random_state=1)
mod_clf = MLPClassifier(random_state=1, max_iter=50).fit(X_train, Y_train)
#print(clf.predict_proba(X_test[:1]))

predict_mlp= mod_clf.predict(X_test)

print("score:",mod_clf.score(X_test, Y_test)*100)
#print("Coef:",mod_clf.coef_)

print("Matrice:\n", metrics.confusion_matrix(Y_test,predict_mlp))

print(classification_report(Y_test,predict_mlp))



score: 98.83408071748879
Matrice:
 [[965   1]
 [ 12 137]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       966
           1       0.99      0.92      0.95       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



## **3) Veille cross validation https://scikit-learn.org/stable/modules/cross_validation.html**  

Utilisez la méthode de ShuffleSplit pour construire vos jeux de données permettant la validation croisée.

In [245]:
scores= cross_val_score(mod_svc, Xstpwrd, validsms, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
result_1 = f1_score(y_true=Y_test, y_pred=predict_svc, labels=None, average="weighted") 
print("formula 1 score!!!:",result_1)
print(classification_report(Y_test,predict_svc))

print(scores)

# Avec shuffle
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scores= cross_val_score(mod_svc, Xstpwrd, validsms, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
result_1 = f1_score(y_true=Y_test, y_pred=predict_svc, labels=None, average="weighted") 
print("formula 1 score!!!:",result_1)
print(scores)


# TEST SHUFFLE ET BOUCLES!


classification= svm.LinearSVC()
mod_svc= classification.fit(X_train,Y_train)

predict_svc = mod_svc.predict(X_test)
print("score:",mod_svc.score(X_test, Y_test)*100)
print("Coef:",mod_svc.coef_)
print("Matrice:\n", metrics.confusion_matrix(Y_test,predict_svc))
print(classification_report(Y_test,predict_svc))

Accuracy: 0.98 (+/- 0.01)
formula 1 score!!!: 0.9806163242065248
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.87      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

[0.98026906 0.9838565  0.97845601 0.97486535 0.97755835]
Accuracy: 0.98 (+/- 0.00)
formula 1 score!!!: 0.9806163242065248
[0.98295964 0.97847534 0.98026906 0.98295964 0.97757848]
score: 98.11659192825111
Coef: [[ 0.47655394  0.7892844  -0.02074406 ... -0.00828722  0.16700293
   0.        ]]
Matrice:
 [[965   1]
 [ 20 129]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.87      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted av

In [247]:
rs = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
#validsms=[1 if x== False else 0 for x in list(sms["ham_or_spam"])]
#corpus=sms["texte"]

for train_index, test_index in rs.split(Xstpwrd):
    X_test=Xstpwrd[test_index]
    X_train=Xstpwrd[train_index]
    #print(train_index," - ",test_index)
    #ytrain=validsms[train_index] #génère l'erreur only integer scalar arrays can be converted to a scalar index
    ytrain= [validsms[i] for i in train_index]
    ytest= [validsms[i] for i in test_index]
    print(ytrain)
    #print(y_train)
    #print(X_train)
    classification= svm.LinearSVC()
    mod_svc= classification.fit(X_train,ytrain)
    predict_svc = mod_svc.predict(X_test)

    scores= cross_val_score(mod_svc, X_test, ytest)

    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    result_1 = f1_score(y_true=Y_test, y_pred=predict_svc, labels=None, average="weighted") 
    print("formula 1 score!!!:",result_1)
    print("Matrice:\n", metrics.confusion_matrix(ytest,predict_svc))
    print(classification_report(y_test,predict_svc))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

Accuracy: 0.95 (+/- 0.03)
formula 1 score!!!: 0.7844994898806559
Matrice:
 [[975   0]
 [ 19 121]]
              precision    recall  f1-score   support

           0       0.87      0.90      0.88       966
           1       0.17      0.13      0.15       149

    accuracy                           0.79      1115
   macro avg       0.52      0.51      0.52      1115
weighted avg       0.78      0.79      0.78      1115

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [138]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scores= cross_val_score(mod_lr, Xstpwrd, validsms, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
result_1 = f1_score(y_true=Y_test, y_pred=predict_lr, labels=None, average="weighted") 
print("formula 1 score!!!:",result_1)

print(scores)

Accuracy: 0.96 (+/- 0.01)
formula 1 score!!!: 0.9556303980094026
[0.95515695 0.94887892 0.95964126 0.96412556 0.95515695]


In [139]:
# ATTENTION TRÉS LONG !!!
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scores= cross_val_score(mod_clf, Xstpwrd, validsms, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
result_1 = f1_score(y_true=Y_test, y_pred=predict_mlp, labels=None, average="weighted") 
print("formula 1 score!!!:",result_1)
print(scores)

Accuracy: 0.98 (+/- 0.01)
formula 1 score!!!: 0.9881503765425436
[0.98654709 0.98116592 0.97755835 0.98025135 0.97935368]


In [195]:
# TEST SHUFFLE ET BOUCLES!
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
#validsms=[1 if x== False else 0 for x in list(sms["ham_or_spam"])]
#corpus=sms["texte"]
y_train=[]
y_test=[]

for train_index, test_index in rs.split(Xstpwrd):
    X_test=Xstpwrd[test_index]
    X_train=Xstpwrd[train_index]
    #print(train_index," - ",test_index)
    for i in train_index:
        y_train.append(validsms[i])
    #print(y_train)
    for i in test_index:
        y_test.append(validsms[i])
    #print(y_test)
    #print("TRAIN:", X_train, "TEST:", X_test)
    #print("Reconst Y: ", y_train= validsms[train_index])
    #print("Reconst Y: ", y_test= validsms[test_index])


## **4) Train**  

Fit the models and compare the performance in a table that show

    the mean of the f1 score
    the standard deviation of the f1 score

Qu'observez vous ?

## **Hors brief, petit code sympa à conserver**

In [8]:
import numpy as np

app = 'Anthony Constant Joshua Fatima Julien Bassem Caroline Dan Ines Nidhal Sacia Xavier Roger Hachem Jean-Pierre Myriam Ludo Olivier Pierre-Etienne Wiem Cecilia'.split()
np.random.seed(1)
rapp = [app[i] for i in np.random.choice(21, 21, replace=False)] #replace à false permet d'avoir un "tirage sans replacement"
for i in range(5):
    print(rapp[4*i:4*i+4] + ([rapp[-1]] if i == 4 else []))

['Myriam', 'Sacia', 'Fatima', 'Pierre-Etienne']
['Olivier', 'Jean-Pierre', 'Ludo', 'Julien']
['Joshua', 'Caroline', 'Dan', 'Constant']
['Hachem', 'Anthony', 'Cecilia', 'Wiem']
['Nidhal', 'Ines', 'Roger', 'Xavier', 'Bassem']


In [ ]:
analyse = vectorisation.build_analyzer()
#print(analyse)

In [63]:
from nltk.corpus import stopwords
print(set(stopwords.words('english')))

{'doesn', 'himself', "that'll", 'but', 'after', 'once', 'isn', "shan't", 'or', 'by', 'for', 'doing', 'between', 'weren', "you'd", "you're", 'is', 'has', 'ourselves', 'any', 're', 'they', 'off', 'there', 'now', 'below', 'themselves', 'some', 'mustn', 'with', 'needn', 'ours', "aren't", "you'll", 'yours', 'our', 'being', "weren't", "couldn't", 'be', 'shouldn', 'when', 'who', 'then', 'through', 'in', 'few', 'i', 'yourselves', 'into', 'just', "hasn't", 'him', "mustn't", 'where', 'd', 'against', 'than', 'and', 'up', 'wasn', 'she', 'ain', 'yourself', 'hers', 'out', 'until', 'my', 'does', 'itself', 'of', 'these', 'on', 't', "doesn't", "won't", 'you', 'at', 've', 'nor', 'them', 'only', 'should', "wouldn't", 'am', 'whom', 'if', 'as', 'their', "isn't", 'most', 'it', 'about', 'll', 'had', 'are', 'from', 'how', 'under', 'won', 'her', 'couldn', 'aren', 'haven', 'what', 'don', 'why', 'herself', 'were', 'such', 'myself', 'same', 'do', 'above', 'm', 'was', "you've", 'we', 'he', 'can', 'his', 'an', 'oth